In response to reviewer #2 suggestion to add more downstream analyses options. 

Take the L-R factor loading values and run an enrichment analysis

In [11]:
import pandas as pd
import numpy as np

import cell2cell as c2c

import os
import sys
sys.path.insert(1, os.path.abspath(os.curdir))
from utility import get_joint_loadings

seed = 888
np.random.seed(seed)
data_path = '/data3/hratch/tc2c_analyses_1/natcomm_revisions/'

In [6]:
# read in loadings (CodeOcean script 06, results/COVID-19/BALF-loadings.xlsx)

balf_tensor = c2c.io.load_variable_with_pickle(data_path + 'processed/Tensor-BALF.pkl')

In [17]:
# get the joint distribution between sender cells and LR pair loadings for factor 8
joint_dis_send = get_joint_loadings(balf_tensor, dim1 = 'LRs', dim2 = 'Sender', 
                              factor = 8)

we are interested in macrophages, and since these are senders, we can assume the joint distribution score represents the ligands:

In [30]:
cell_type = 'Macrophages'
macrophage_scores = pd.DataFrame(data={'score': joint_dis_send[cell_type].tolist(),
                                         'ligand': [lr_pair.split('^')[0] for lr_pair in joint_dis_send.index]})
macrophage_scores = macrophage_scores.groupby('ligand').mean().sort_values(by = 'score', ascending = False)

In [40]:
sns.distplot(macrophage_scores)

NameError: name 'sns' is not defined